### Importación de librerías

In [1]:
import numpy as np
import pandas as pd

### Lectura del dataset de transacciones

In [2]:
a=pd.read_csv('dt_trxpse_personas_2016_2018_muestra_adjt.csv', error_bad_lines=False,warn_bad_lines=False, header= None)
a=a.rename(columns={0:"id_trn_ach",1:"id_cliente",2:"fecha",
                  3:"hora",4:"valor_trx",5:"ref1",6:"ref2",
                  7:"ref3",8:"sector",9:"subsector",10:"descripcion",})  # asignacion de encabezados
print (a.shape)

(11806137, 11)


#### Categorías

Las categorías definidas se basan en la columna subsector, fueron definidas y mapeadas de forma manual, algunos aspectos importantes a tener en cuenta, puede haber sesgo en las clases, al tener pocos ejemplos por clase, también se encontró que aunque hay registros con el mismo subsector difieren mucho entre si basados en la descripción, aunque para tener una mejor anotación del data set se debería explorar la descripción, puede llegar a ser algo dispendioso, también puede ser un problema, tener muchas clases y no tener variables que permitan una buena discriminación de los registros.

1. Pago de Obligaciones
2. Personal y Hogar
3. Otros

### Muestra del Dataset

In [4]:
a.head(10)

,id_trn_ach,id_cliente,fecha,hora,valor_trx,ref1,ref2,ref3,sector,subsector,descripcion
0,230435642,3,20161207.0,113451,2122392.51,CC,NaN,NaN,\N,\N,\N
1,222356110,10,20161016.0,3424,148438.37,Referencia: Contrato: Valor:,CC,NaN,\N,\N,\N
2,309137749,10,20180120.0,195042,94025.19,CC,NaN,NaN,\N,\N,\N
3,324614737,10,20180326.0,192146,94430.07000000001,CC,NaN,NaN,\N,\N,\N
4,235344690,18,20170106.0,201317,670645.5699999999,MEDICINA PREPAGADA COLSANITAS,CE,NaN,\N,\N,\N
5,320049316,18,20180307.0,143513,706933.7,RECAUDO COLSANITAS,CC,NaN,\N,\N,\N
6,230519178,19,20161207.0,155840,306773.79,CC,NaN,NaN,\N,\N,\N
7,241307506,19,20170210.0,142809,701067.98,Pago de la Planilla Cesantias,CEDULA DE CIUDADANIA,NaN,SERVICIOS FINANCIEROS,OTROS SERVICIOS FINANCIEROS,Otras actividades auxiliares de las actividade...
8,316193109,24,20180220.0,153148,767571.01,CC,NaN,NaN,\N,\N,\N
9,282076350,26,20170914.0,70026,100277.45,Pago de la factura # CONJUNTO RESIDENCIAL PIET...,IDC,NaN,\N,\N,\N


### Limpieza de registros
Se plantea realizar un etiquetado de los registros basándose en su subsector, es por esto que se filtrara todos los registros que no tengan esta información.

In [5]:
a=a[a.subsector!='\\N']
a=a[a.subsector.notna()]
a=a.reset_index(drop=True)

Se eliminan las columnas que ya no son necesarias

In [11]:
del a["ref1"]
del a["ref2"]
del a["ref3"]
del a["sector"]
del a["descripcion"]

Muestra del Dataset resultante

In [12]:
a.head()

,id_trn_ach,id_cliente,fecha,hora,valor_trx,subsector
0,241307506,19,20170210.0,142809,701067.98,OTROS SERVICIOS FINANCIEROS
1,359125394,35,20180808.0,122953,246964.14,BANCOS
2,285847659,40,20171003.0,151703,192491.73,BANCOS
3,319249942,45,20180305.0,111143,49520.98,MUNICIPIOS
4,335337578,45,20180507.0,145248,431146.54,BANCOS


### Definición del diccionario subsector-etiqueta
Este es el diccionario que mapea cada uno de los subsectores a su clase correspondiente

In [13]:
dic={'ADMINISTRACIÓN CENTRAL':1,
     'AGROQUIMICOS':3,
     'ALIMENTOS CONCENTRADOS':3,
     'ARROZ':3, 
     'ASEO':3, 
     'AUTOMOTORES':2, 
     'BANCOS':1, 
     'CAJAS DE COMPENSACIÓN':2,
     'COMERCIO DE REPUESTOS':3,
     'COMERCIO DE VARIEDADES Y VESTUARIO':2,
     'COMISIONISTAS DE BOLSA':3, 
     'COMPANÍAS DE FINANCIAMIENTO COMERCIAL':1,
     'DEPARTAMENTOS':1, 
     'DISTRIBUIDORES PRODUCTOS CONSUMO MASIVO':3,
     'EDIFICACIONES':2, 
     'ELECTRICIDAD':2, 
     'EPS Y SALUD PREPAGADA (SALUD)':2,
     'ESTABLECIMIENTOS EDUCATIVOS':2, 
     'FONDOS DE PENSIONES':2,
     'IMPRENTAS Y EDICION':3, 
     'IPS':2, 
     'MADERA Y MUEBLES':2, 
     'MEDIOS':3,
     'MUNICIPIOS':1, 
     'OBRAS DE INFRAESTRUCTURA':1,
     'OTRAS ACTIVIDADES MINERAS':3, 
     'OTROS MANUFACTURA':3,
     'OTROS SERVICIOS FINANCIEROS':1, 
     'PERSONAS':2, 
     'PLASTICO':3,
     'PRODUCTOS CERAMICOS':3, 
     'PRODUCTOS METALICOS':3, 
     'SEGUROS':2,
     'SERVICIOS A EMPRESAS':3, 
     'SERVICIOS A PERSONAS':2, 
     'TELEFONIA FIJA':2,
     'TEXTILES':3, 
     'TRANSPORTE AEREO':2, 
     'TRANSPORTE TERRESTRE':2,
     'VALOR AGREGADO':2}

### Mapeo de etiquetas

se agrega el campo etiqueta basandose en su subsector y se elimina el campo sub sector del registro

In [14]:
a["etiqueta"]=a.subsector.map(dic)
del a["subsector"]

muestra

In [15]:
a.head()

,id_trn_ach,id_cliente,fecha,hora,valor_trx,etiqueta
0,241307506,19,20170210.0,142809,701067.98,1
1,359125394,35,20180808.0,122953,246964.14,1
2,285847659,40,20171003.0,151703,192491.73,1
3,319249942,45,20180305.0,111143,49520.98,1
4,335337578,45,20180507.0,145248,431146.54,1


muestra  registros por categoría

In [16]:
a.etiqueta.value_counts()

2    1748550
1    1488462
3      37243
Name: etiqueta, dtype: int64

Guardado del dataset

In [17]:
a.to_csv("Dataset_etiquetado.csv",index=False)